# 01 - Matches par équipe (fetch/debug)
Scrape les matches via l'API OpenDota `/teams/{team_id}/matches`, sauvegarde brut, et aperçu rapide pour debug.
- Clé via `OPENDOTA_KEY` (.env).
- Helpers réutilisables dans `src.dota_data.api`.


In [1]:
from pathlib import Path
import sys

def _find_root():
    candidates = [Path.cwd()] + list(Path.cwd().parents[:4])
    for cand in candidates:
        if (cand / "src").exists():
            return cand
    return Path.cwd()

ROOT = _find_root()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))
print(f"Using project root: {ROOT}")


Using project root: /home/ju/Documents/Dev/Dota-Datas


In [2]:
from pathlib import Path
import json
import pandas as pd
from dotenv import load_dotenv

from src.dota_data.api import (
    load_api_key,
    build_session,
    load_team_list,
    fetch_team_matches,
    annotate_matches_with_team,
    write_json,
)

load_dotenv()
teams = load_team_list(ROOT / "data/teams_to_look.csv")
api_key = load_api_key(load_env_file=False)
session = build_session(api_key)
print(f"Teams chargées: {len(teams)}")


Teams chargées: 24


In [3]:
team_matches = []
for team in teams:
    matches = fetch_team_matches(team["TeamID"], session=session)
    team_matches.append({"team": team, "matches": matches})

total_matches = sum(len(t["matches"] ) for t in team_matches)
print(f"Matches récupérés: {total_matches} sur {len(teams)} équipes")


Matches récupérés: 19161 sur 24 équipes


In [4]:
flat_rows = []
for entry in team_matches:
    flat_rows.extend(annotate_matches_with_team(entry["matches"], entry["team"]["TeamID"], entry["team"]["TeamName"]))

matches_df = pd.DataFrame(flat_rows)
if not matches_df.empty:
    matches_df["start_dt"] = pd.to_datetime(matches_df["start_time"], unit="s")
matches_df.head()


,match_id,radiant_win,radiant_score,dire_score,radiant,duration,start_time,leagueid,league_name,cluster,opposing_team_id,opposing_team_name,opposing_team_logo,_source_team_id,_source_team_name,start_dt
0,8547022523,True,23,19,False,2312,1762520179,17419,SLAM IV,153,9303484,HEROIC,https://cdn.steamusercontent.com/ugc/247198417...,9467224,Aurora Gaming,2025-11-07 12:56:19
1,8546927307,True,31,4,False,1995,1762515793,17419,SLAM IV,153,9303484,HEROIC,https://cdn.steamusercontent.com/ugc/247198417...,9467224,Aurora Gaming,2025-11-07 11:43:13
2,8531221816,True,23,20,False,2310,1761580800,18863,FISSURE PLAYGROUND 2,274,9338413,MOUZ,https://cdn.steamusercontent.com/ugc/149367842...,9467224,Aurora Gaming,2025-10-27 16:00:00
3,8531103673,False,13,31,False,2555,1761576420,18863,FISSURE PLAYGROUND 2,273,9338413,MOUZ,https://cdn.steamusercontent.com/ugc/149367842...,9467224,Aurora Gaming,2025-10-27 14:47:00
4,8531000461,True,23,12,False,1749,1761572880,18863,FISSURE PLAYGROUND 2,272,9338413,MOUZ,https://cdn.steamusercontent.com/ugc/149367842...,9467224,Aurora Gaming,2025-10-27 13:48:00


In [5]:
out_path = Path("data/interim/team_matches_raw.json")
write_json(team_matches, out_path)
print(f"Sauvegardé {out_path} ({out_path.stat().st_size/1024:.1f} KB)")


Sauvegardé data/interim/team_matches_raw.json (9928.1 KB)


In [6]:
# Debug rapide: volume par équipe et stats de base
if not matches_df.empty:
    display(matches_df.groupby("_source_team_name")["match_id"].nunique().sort_values(ascending=False))
    display(matches_df.describe(include="all"))


_source_team_name
Team Liquid        2898
Natus Vincere      2545
OG                 2072
Team Spirit        1358
BOOM Esports       1206
Tundra Esports     1151
Nigma Galaxy       1055
BetBoom Team       1002
Xtreme Gaming       907
Team Falcons        667
MOUZ                618
Passion UA          615
1w Team             589
HEROIC              521
Aurora Gaming       374
Yakult Brothers     338
Runa Team           325
PARIVISION          321
Team Tidebound      246
Pipsqueak+4         125
Team Yandex         125
Virtus.pro           49
Amaru Gaming         42
GamerLegion          12
Name: match_id, dtype: int64

,match_id,radiant_win,radiant_score,dire_score,radiant,duration,start_time,leagueid,league_name,cluster,opposing_team_id,opposing_team_name,opposing_team_logo,_source_team_id,_source_team_name,start_dt
count,1.916100e+04,19161,19161.000000,19161.000000,19161,19161.000000,1.916100e+04,19161.000000,19161,19161.000000,1.916100e+04,19161,18910,1.916100e+04,19161,19161
unique,NaN,2,NaN,NaN,2,NaN,NaN,NaN,434,NaN,NaN,694,700,NaN,24,NaN
top,NaN,True,NaN,NaN,False,NaN,NaN,NaN,European Pro League 2024-2025 Season,NaN,NaN,Team Liquid,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,NaN,Team Liquid,NaN
freq,NaN,9609,NaN,NaN,9629,NaN,NaN,NaN,337,NaN,NaN,783,783,NaN,2898,NaN
mean,6.405055e+09,NaN,22.161735,22.210271,NaN,2320.592714,1.651983e+09,14073.836386,NaN,212.318303,5.894235e+06,NaN,NaN,5.446684e+06,NaN,2022-05-08 04:03:10.541986560
min,2.242330e+07,NaN,0.000000,0.000000,NaN,428.000000,1.340648e+09,4.000000,NaN,1.000000,2.000000e+00,NaN,NaN,3.600000e+01,NaN,2012-06-25 18:09:47
25%,5.464367e+09,NaN,13.000000,12.000000,NaN,1871.000000,1.591816e+09,12113.000000,NaN,152.000000,1.846548e+06,NaN,NaN,2.163000e+03,NaN,2020-06-10 19:08:59
50%,7.462720e+09,NaN,23.000000,23.000000,NaN,2247.000000,1.701180e+09,15898.000000,NaN,224.000000,8.126892e+06,NaN,NaN,7.554697e+06,NaN,2023-11-28 14:07:59
75%,8.171590e+09,NaN,32.000000,32.000000,NaN,2669.000000,1.739423e+09,17509.000000,NaN,273.000000,9.175409e+06,NaN,NaN,9.247354e+06,NaN,2025-02-13 05:00:02
max,8.594671e+09,NaN,71.000000,80.000000,NaN,7065.000000,1.765120e+09,65013.000000,NaN,412.000000,9.980558e+06,NaN,NaN,9.964962e+06,NaN,2025-12-07 15:00:39
